In [ ]:
!pip install PyMySQL
!pip install selenium

In [1]:
import requests
from bs4 import BeautifulSoup
import pymysql
from pandas import DataFrame
from  selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time as time
import getpass
import urllib.request as req
import random
import datetime
from dateutil.parser import parse
 
 
from time import sleep
 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
path = "C:\\Users\\user\Downloads\chromedriver_win32\chromedriver.exe" # Put the directory of Chromedriver.exe
driver = webdriver.Chrome(path)
press_nm = input('검색할 언론사  : ')
query = input('검색할 키워드  : ')

if(press_nm == "BBC"):
    news_url = 'https://www.bbc.co.uk/search?q={}'.format(query)
elif(press_nm == "CNN"):
    news_url = 'https://edition.cnn.com/search?q={}'.format(query)
elif(press_nm == "아사이"):
    news_url = 'https://www.asahi.com/'
elif(press_nm == "theguardian"):
    news_url = 'https://www.google.co.kr'
    
driver.get(news_url)
driver.maximize_window()

time.sleep(1)

검색할 언론사  : theguardian
검색할 키워드  : vaccine


# CNN

In [ ]:
press_dict = {'title':[],
            'date': [],
            'text': [],
            }

In [ ]:
# CNN 크롤링
page_num = 0
news_url = 'https://edition.cnn.com/search?q={}&size=10&from={}&page={}'.format(query,  page_num*10, page_num+1)

while(page_num < 960):
    for i in range (0,10):
        title_info = driver.find_elements_by_css_selector('h3.cnn-search__result-headline')[i].text
        press_dict['title'].append(title_info)
    
        time_info = driver.find_elements_by_css_selector('div.cnn-search__result-publish-date')[i].text
        press_dict['date'].append(time_info)
    
        text_raw = driver.find_elements_by_css_selector('div.cnn-search__result-body')[i].text
        press_dict['text'].append(text_raw)

    page_num += 1
    news_url = 'https://edition.cnn.com/search?q={}&size=10&from={}&page={}'.format(query,  page_num*10, page_num+1)
    driver.get(news_url)
    time.sleep(2)
    

In [ ]:
press_df = pd.DataFrame.from_dict(press_dict)

press_df.to_csv("data_CNN_vaccine.csv", encoding = "utf-8")

# BBC

In [ ]:
press_dict = {'title':[],
              'location':[],
            'date': [],
            'text': [],
            }

In [ ]:
# BBC 크롤링
for i in range(20,30):
    num = 0    
    while num != 10:
        url = "https://www.bbc.co.uk/search?q=vaccine&page={}".format(i)
        driver.get(url)       
        text_result = ""
        text_num = 0
        
        if(driver.find_elements_by_css_selector('dd.ssrcss-m5j4pi-MetadataContent.e1ojgjhb0')[num*3+1].text=="News"):        
            # 제목 수집
            title_info = driver.find_elements_by_css_selector('p.ssrcss-6arcww-PromoHeadline.e1f5wbog4')[num].text
            BBC_dict['title'].append(title_info)
        
        
            # 위치 수집
            location_info = driver.find_elements_by_css_selector('dd.ssrcss-m5j4pi-MetadataContent.e1ojgjhb0')[num*3+2].text
            BBC_dict['location'].append(location_info)
            
            # 창 이동
            driver.find_elements_by_css_selector('a.ssrcss-bxvqns-PromoLink.e1f5wbog0')[num].click()
            
            # 날짜 수집
            time_raw = driver.find_element_by_css_selector('time')
            time_info = pd.to_datetime(time_raw.get_attribute('datetime'))
            BBC_dict['date'].append(time_info)
            
            # 텍스트 수집
            while True:
                try:
                    text_raw = driver.find_elements_by_css_selector('div.ssrcss-18snukc-RichTextContainer.e5tfeyi1')[text_num].text
                    text_result = text_result + text_raw
                    text_num += 1
                except:
                    break
                    
            BBC_dict['text'].append(text_result)
            
            driver.back()
        
        num += 1
        
        if(i%5 == 0 and num == 1):
            test = pd.DataFrame.from_dict(BBC_dict)
            test.to_csv("data_BBC_vaccine3.csv", encoding = "utf-8")

# Asahi

In [ ]:
press_dict = {'title':[],
            'date': [],
            'text': [],
            }

In [ ]:
# 아사히 크롤링
page_num = 394

# 키워드[백신] 
# Keywords= 뒷부분이 인코딩이 일본어로 안됨
news_url = 'https://sitesearch.asahi.com/sitesearch/?Keywords=%E3%83%AF%E3%82%AF%E3%83%81%E3%83%B3&Searchsubmit2=%E6%A4%9C%E7%B4%A2&Searchsubmit=%E6%A4%9C%E7%B4%A2&iref=pc_ss_date_btn3&sort=&start={}'.format(page_num*20)

driver.get(news_url)
time.sleep(1)
    
while page_num * 20 <= 8300:
    table = driver.find_element_by_xpath('//ul[@class="ListBlock"]')
    li_list = table.find_elements_by_xpath('./li[@class = "digital"]')
    #li_list2 = table.find_elements_by_xpath('./li[@class = "shimbun"]')
    title_list = [li.find_element_by_xpath('.//span[@class="SearchResult_Headline"]') for li in li_list]
    #title_list2 = [li2.find_element_by_xpath('.//span[@class="SearchResult_Headline"]') for li2 in li_list2]
    date_list = [title.find_element_by_xpath('.//span[@class="Date"]') for title in title_list]
    #date_list2 = [title2.find_element_by_xpath('.//span[@class="Date"]') for title2 in title_list2]
    
    for n in title_list:
        press_dict['title'].append(n.text)
        
    for m in date_list:
        press_dict['date'].append(m.text)
        
    for k in range(0, len(title_list)):
        try:
            news_url = driver.find_elements_by_xpath('.//li[@class="digital"]/a')[k].get_attribute('href')
            driver.get(news_url)
            time.sleep(1)        
        
            #text_info = driver.find_element_by_xpath('.//div[@class="_2rW9J"]').text
            text_info = driver.find_element_by_xpath('.//div[@class="ArticleText"]').text
            press_dict['text'].append(text_info)
            driver.back()
            time.sleep(1)
        except:
            press_dict['text'].append("0")
    
    # 신문도 크롤링할 때
    '''for n2 in title_list2:
        press_dict['title'].append(n2.text)
        
    for m2 in date_list2:
        press_dict['date'].append(m2.text)
        
    for k2 in range(0, len(title_list2)):
        driver.find_elements_by_css_selector('li.shimbun')[k2].click()
        time.sleep(1.5)        
        
        text_raw = driver.find_element_by_css_selector('div._2rW9J').text
        press_dict['text'].append(text_raw)
                
        driver.back()
        time.sleep(1)'''
    
        
###################창 바꿈####################
    page_num += 1
            
    news_url = 'https://sitesearch.asahi.com/sitesearch/?Keywords=%E3%83%AF%E3%82%AF%E3%83%81%E3%83%B3&Searchsubmit2=%E6%A4%9C%E7%B4%A2&Searchsubmit=%E6%A4%9C%E7%B4%A2&iref=pc_ss_date_btn3&sort=&start={}'.format(page_num*20)
    driver.get(news_url)
    time.sleep(1.5)

In [ ]:
press_df = pd.DataFrame.from_dict(press_dict)
press_df

press_df.to_csv("data_asahi_vaccine.csv", encoding = "utf-8")

# The Guardian

In [3]:
press_dict = {'title':[],
            'date': [],
            'text': [],
            }

In [4]:
# The Guardian
element_search = driver.find_element_by_name("q")
element_search.send_keys('site:www.theguardian.com {}'.format(query))
element_search.send_keys(Keys.RETURN)
time.sleep(1)

In [ ]:
table = driver.find_element_by_xpath('//div[@class="v7W49e"]')
li_list = table.find_elements_by_xpath('./div[@class = "g"]')
title_list = [li.find_element_by_xpath('.//h3[@class="LC20lb DKV0Md"]') for li in li_list]
date_list = [li.find_element_by_xpath('.//span[@class="MUxGbd wuQ4Ob WZ8Tjf"]') for li in li_list]

In [ ]:
for k in range(0, len(title_list)):
    press_dict['title'].append(title_list[k].text)
    press_dict['date'].append(date_list[k].text)
    
    #news_url = driver.find_elements_by_xpath('.//div[@class="yuRUbf"]/a')[0].get_attribute('href')
    #driver.get(news_url)
    
    #text_info = driver.find_element_by_xpath('.//div[@class="article-body-commercial-selector article-body-viewer-selector  dcr-ucgxn1"]').text
    #press_dict['text'].append(text_info)

In [15]:
i = 0
while True:
        table = driver.find_element_by_xpath('//div[@class="v7W49e"]')
        li_list = table.find_elements_by_xpath('./div[@class = "g"]')
        title_list = [li.find_element_by_xpath('.//h3[@class="LC20lb DKV0Md"]') for li in li_list]
        try:
            date_list = [li.find_element_by_xpath('//span[@class="MUxGbd wuQ4Ob WZ8Tjf"]') for li in li_list]
        except:
            date_list = [li.find_element_by_xpath('//div[@class="fG8Fp uo4vr"]') for li in li_list]

        for k in range(0, len(title_list)):
            press_dict['title'].append(title_list[k].text)
            press_dict['date'].append(date_list[k].text)
    
        for k in range(0, len(title_list)):
            news_url = driver.find_elements_by_xpath('.//div[@class="yuRUbf"]/a')[k].get_attribute('href')
            driver.get(news_url)
            time.sleep(1)
    
            try:
                text_info = driver.find_element_by_xpath('.//div[@class="article-body-commercial-selector article-body-viewer-selector  dcr-ucgxn1"]').text
                press_dict['text'].append(text_info)
    
            except:
                press_dict['text'].append("0")
    
            driver.back()
            time.sleep(1)

    
        driver.find_elements_by_css_selector('td.d6cvqb.BBwThe')[1].click()
        time.sleep(1)
        i+=1
    
        if(i == 100):
            break

In [16]:
press_dict

{'title': ['Pregnant women are being turned away from UK Covid ...',
  'Covid-19 vaccine Australia rollout tracker by state: total number',
  'Vaccines and immunisation | Society | The Guardian',
  'Thousands of AstraZeneca Covid vaccine doses going to ...',
  "FDA advisers recommend approval of Pfizer's Covid vaccine ...",
  'Which protects you more against Covid – vaccination or prior ...',
  "See Australia's state and territory Covid vaccination rollouts ...",
  'Australian GPs offered bribes for fake Covid vaccine ...',
  'Leos are most likely to get vaccinated, say Utah officials. Is it ...',
  'News Corp Australia to introduce Covid vaccine mandate for ...',
  'Ministers to ramp up Covid vaccine rollout as hospitalisations ...',
  'NSW Covid vaccination rate by postcode - The Guardian',
  "Australian business groups lament 'nightmare' of states ...",
  "New Zealand's weird and wonderful vaccine rollout - The ...",
  "Rising number of anti-vaxxers 'willing to take direct action ..

# Saving

In [17]:
press_df = pd.DataFrame.from_dict(press_dict)
press_df

,title,date,text
0,Pregnant women are being turned away from UK C...,1일 전 —,Pregnant women are being turned away from Covi...
1,Covid-19 vaccine Australia rollout tracker by ...,1일 전 —,0
2,Vaccines and immunisation | Society | The Guar...,1일 전 —,0
3,Thousands of AstraZeneca Covid vaccine doses g...,1일 전 —,"Almost 1,000 Covid vaccination providers are d..."
4,FDA advisers recommend approval of Pfizer's Co...,1일 전 —,Independent advisers for the US Food and Drug ...
...,...,...,...
891,Under-30s in UK should be offered alternative ...,7시간 전 —,0
892,'That was easy': Kamala Harris receives Covid-...,7시간 전 —,0
893,New Zealand's South Island records first Covid...,7시간 전 —,New Zealand’s South Island has recorded its fi...
894,Schwarzenegger receives Covid-19 vaccine – vid...,7시간 전 —,0


In [22]:
press_df.to_csv("data_theguardian_vaccine.csv", encoding = "utf-8")

In [21]:
pd.set_option('display.max_row', None)
press_df

,title,date,text
0,Pregnant women are being turned away from UK C...,1일 전 —,Pregnant women are being turned away from Covi...
1,Covid-19 vaccine Australia rollout tracker by ...,1일 전 —,0
2,Vaccines and immunisation | Society | The Guar...,1일 전 —,0
3,Thousands of AstraZeneca Covid vaccine doses g...,1일 전 —,"Almost 1,000 Covid vaccination providers are d..."
4,FDA advisers recommend approval of Pfizer's Co...,1일 전 —,Independent advisers for the US Food and Drug ...
5,Which protects you more against Covid – vaccin...,1일 전 —,Australians could soon be able to access a Cov...
6,See Australia's state and territory Covid vacc...,1일 전 —,0
7,Australian GPs offered bribes for fake Covid v...,1일 전 —,GPs are being offered thousands of dollars in ...
8,"Leos are most likely to get vaccinated, say Ut...",1일 전 —,E\nxciting news for people who believe in scie...
9,News Corp Australia to introduce Covid vaccine...,1일 전 —,News Corp Australia will not allow anyone who ...
